In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import os

In [ ]:
df = pd.read_csv("Lines_v3.csv")

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
quotes = df["quote"].astype(str).tolist()


In [ ]:
embeddings = model.encode(quotes, show_progress_bar=True)
embeddings = np.array(embeddings)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [66]:
sent_embeddings = embeddings[0:31]

In [ ]:
os.makedirs("projector_files", exist_ok=True)
np.savetxt("projector_files/vectors_2.tsv", embeddings, delimiter="\t")

In [67]:
os.makedirs("projector_files", exist_ok=True)
np.savetxt("projector_files/vectors_4.tsv", sent_embeddings, delimiter="\t")

In [ ]:
quotes = embeddings[0:31]
concepts = embeddings[31:52]

In [ ]:
import torch

quote_vectors = torch.tensor(quotes)      # shape: Q x 384
concept_vectors = torch.tensor(concepts)

tensor([[ 0.0100, -0.0772,  0.0149,  ..., -0.1419,  0.0052,  0.0337],
        [-0.0156,  0.0075, -0.0493,  ..., -0.0393,  0.0114,  0.0075],
        [ 0.0223,  0.0500,  0.0010,  ...,  0.0176,  0.0427, -0.0642],
        ...,
        [-0.0034,  0.0345, -0.0307,  ...,  0.0241, -0.0038, -0.0133],
        [ 0.0666,  0.0411,  0.0978,  ...,  0.0381, -0.0666, -0.0679],
        [ 0.1101, -0.0214,  0.0560,  ..., -0.0187, -0.0933, -0.0593]])

In [ ]:
with open("projector_files/metadata_2.tsv", "w", encoding="utf-8") as f:
    f.write("work\thero\ttheme\tquote\n")
    for _, row in df.iterrows():
        quote_clean = str(row["quote"]).replace("\t"," ").replace("\n"," ")
        f.write(f"{row['work']}\t{row['hero']}\t{row['theme']}\t{quote_clean}\n")

print("Done! Upload vectors.tsv and metadata.tsv to projector.tensorflow.org")

Done! Upload vectors.tsv and metadata.tsv to projector.tensorflow.org


In [ ]:
quotes = embeddings[0:31]
concepts = embeddings[30:52]

In [37]:
concept_labels = df[31:52]["quote"].tolist()

In [52]:
from sentence_transformers.util import cos_sim

similarity_matrix = cos_sim(quote_vectors, concept_vectors)

In [55]:
nearest = torch.argmax(similarity_matrix, dim=1)


In [53]:
nearest

tensor([ 4,  7,  7, 18, 16,  6,  7, 17,  7,  7,  7,  7,  5, 17,  5,  7,  2,  8,
         6,  8, 19, 14,  6,  4,  6, 13,  6,  6, 17, 13,  9])

In [58]:
closest_concepts = [concept_labels[idx.item()] for idx in nearest]


In [59]:
closest_concepts

['heroic greatness',
 'divine power',
 'divine power',
 'human limitation',
 'impossible feats',
 'trained skill',
 'divine power',
 'divine intervention',
 'divine power',
 'divine power',
 'divine power',
 'divine power',
 'tragic greatness',
 'divine intervention',
 'tragic greatness',
 'divine power',
 'divine greatness',
 'inner strength',
 'trained skill',
 'inner strength',
 'overcoming doubt',
 'effortless mastery',
 'trained skill',
 'heroic greatness',
 'trained skill',
 'hidden suffering',
 'trained skill',
 'trained skill',
 'divine intervention',
 'hidden suffering',
 'moral duty']

In [60]:
similarity_scores = [
    similarity_matrix[i, nearest[i]].item()
    for i in range(len(nearest))
]


In [61]:
similarity_scores

[0.25434401631355286,
 0.24664582312107086,
 0.3846152126789093,
 0.22063903510570526,
 0.2490011751651764,
 0.18157154321670532,
 0.23403087258338928,
 0.19954359531402588,
 0.2533738315105438,
 0.1716783344745636,
 0.15983131527900696,
 0.21316467225551605,
 0.2704649567604065,
 0.25116029381752014,
 0.21113133430480957,
 0.3735935688018799,
 0.4205889105796814,
 0.3988514542579651,
 0.31499290466308594,
 0.4350925087928772,
 0.2567877173423767,
 0.19701522588729858,
 0.187896728515625,
 0.2808898389339447,
 0.20192545652389526,
 0.21266071498394012,
 0.3239055871963501,
 0.16001468896865845,
 0.155730202794075,
 0.42510199546813965,
 0.23193751275539398]

In [62]:
df_sent = df.iloc[:len(nearest)].copy()


In [63]:
df_sent["closest_concept"] = closest_concepts
df_sent["similarity_score"] = similarity_scores

In [64]:
df_sent

,id,work,hero,theme,quote,closest_concept,similarity_score
0,1,mahabharata,arjuna,mastery,More marvels follow.,heroic greatness,0.254344
1,2,mahabharata,arjuna,skill,Arjuna moves swiftly with the panache and pois...,divine power,0.246646
2,3,mahabharata,arjuna,confidence,"He is master of his body, never doubting that ...",divine power,0.384615
3,4,mahabharata,arjuna,divine_power,He changes size at will.,human limitation,0.220639
4,5,mahabharata,arjuna,divine_power,He conjures mountains.,impossible feats,0.249001
5,6,mahabharata,arjuna,skill,"He shoots an arrow into the sand, making the g...",trained skill,0.181572
6,7,mahabharata,arjuna,divine_power,"He stands with his eyes closed, his lips movin...",divine power,0.234031
7,8,mahabharata,arjuna,skill,He shoots an arrow straight into the sky.,divine intervention,0.199544
8,9,mahabharata,arjuna,divine_power,Slowly it rises.,divine power,0.253374
9,10,mahabharata,arjuna,divine_power,Then it begins to glow.,divine power,0.171678


In [65]:
with open("projector_files/metadata_2.tsv", "w", encoding="utf-8") as f:
    # add the two new columns to the header
    f.write("work\thero\ttheme\tquote\tclosest_concept\tsimilarity_score\n")

    for _, row in df_sent.iterrows():
        quote_clean = str(row["quote"]).replace("\t", " ").replace("\n", " ")
        f.write(
            f"{row['work']}\t"
            f"{row['hero']}\t"
            f"{row['theme']}\t"
            f"{quote_clean}\t"
            f"{row['closest_concept']}\t"
            f"{row['similarity_score']:.4f}\n"
        )

print("Done! Upload vectors.tsv and metadata_2.tsv to projector.tensorflow.org")


Done! Upload vectors.tsv and metadata_2.tsv to projector.tensorflow.org


In [ ]:
concepts

array([[-0.04454196,  0.07048328,  0.04378927, ..., -0.04906201,
        -0.03649219, -0.00304459],
       [ 0.0014107 ,  0.04719924,  0.04457599, ..., -0.02345482,
         0.01536639, -0.00813303],
       [-0.01287239,  0.05529012,  0.05264983, ..., -0.00738572,
         0.02497091, -0.02230209],
       ...,
       [ 0.01873592,  0.09718287,  0.06383316, ..., -0.01790823,
         0.05444858, -0.07150358],
       [ 0.04063177,  0.00808774, -0.03525032, ...,  0.01940689,
        -0.0127061 ,  0.03430426],
       [ 0.01072357,  0.03183746,  0.02661521, ..., -0.04500725,
         0.05559666, -0.02368062]], dtype=float32)

In [ ]:
concept_vectors

tensor([-4.4542e-02,  7.0483e-02,  4.3789e-02, -6.1110e-02, -4.4681e-02,
         5.1069e-02,  1.5919e-01,  2.3005e-02, -5.7847e-02, -2.0551e-02,
         4.1833e-02,  1.7373e-02,  4.2707e-02,  1.6492e-02, -6.5791e-02,
         3.6561e-02,  5.1382e-02,  7.6863e-03, -1.3651e-01, -1.1056e-01,
         2.9575e-02,  2.3798e-02, -6.5157e-03, -1.0788e-02, -1.9096e-02,
         3.1059e-02, -1.4443e-02,  4.6202e-02,  1.0851e-01, -3.1712e-02,
        -3.2896e-02, -2.4266e-02,  9.9635e-02,  1.0415e-02, -7.4975e-02,
         2.8525e-02,  2.3734e-02,  1.8585e-02,  1.0772e-02, -6.1483e-02,
        -1.9630e-02, -5.3529e-02,  3.6460e-02, -7.7210e-02,  1.2852e-02,
        -4.8036e-02,  7.7378e-02, -3.4832e-02, -4.3639e-02, -5.3455e-03,
        -4.1381e-03, -2.6921e-03, -4.2287e-02,  6.8633e-03,  7.0112e-02,
         9.1970e-02,  1.1434e-02, -6.5304e-02, -2.7516e-02, -3.3715e-02,
        -1.7821e-02,  3.6538e-02,  2.5360e-02, -2.7032e-02,  7.5600e-02,
        -2.9878e-02, -4.4879e-02,  1.0764e-01, -1.5

In [42]:
from sentence_transformers.util import cos_sim

similarity_matrix = cos_sim(quote_vectors, concept_vectors)

In [48]:
nearest = torch.argmax(similarity_matrix, dim=1)


In [49]:
nearest

tensor([ 4,  7,  7, 18, 16,  6,  7, 17,  7,  7,  7,  7,  5, 17,  5,  7,  2,  8,
         6,  8, 19, 14,  6,  4,  6, 13,  6,  6, 17, 13,  9])

In [ ]:
top3 = torch.topk(similarity_matrix, k=3, dim=1)


In [ ]:
similarity_matrix

tensor([[ 0.0969,  0.1640,  0.2072,  0.1551,  0.2543,  0.1656,  0.0408,  0.2258,
          0.0561,  0.0845,  0.0739,  0.0185,  0.2056,  0.0651,  0.1612,  0.1140,
          0.1642,  0.1996,  0.1077,  0.0759],
        [ 0.1715,  0.1450,  0.2396,  0.1462,  0.2270,  0.1716,  0.1980,  0.2466,
          0.0808,  0.0766,  0.1635,  0.1683,  0.1753,  0.0877,  0.1309,  0.1332,
          0.1870,  0.1463,  0.0301,  0.0454],
        [ 0.1396,  0.1642,  0.3286,  0.2428,  0.2377,  0.1193,  0.1844,  0.3846,
          0.2177,  0.2049,  0.2297,  0.0122,  0.1736,  0.1516,  0.2623,  0.2800,
          0.2360,  0.2618,  0.2229,  0.2120],
        [-0.0019,  0.0999,  0.0791, -0.0044,  0.0844,  0.0366,  0.2017,  0.1383,
          0.1672, -0.0117,  0.0422,  0.0401,  0.0826,  0.0506,  0.1563,  0.1258,
          0.1482,  0.1756,  0.2206,  0.1308],
        [ 0.1332,  0.1476,  0.2422,  0.0789,  0.2118,  0.1565,  0.2412,  0.2226,
          0.2004,  0.0650,  0.1037, -0.0240,  0.1355,  0.1746,  0.1970,  0.1823,
      

torch.return_types.topk(
values=tensor([[0.3626, 0.3395, 0.3120],
        [0.3160, 0.3060, 0.3052],
        [0.5276, 0.4897, 0.4580],
        [0.2889, 0.2667, 0.2639],
        [0.3536, 0.3354, 0.3255],
        [0.3317, 0.3168, 0.3073],
        [0.2901, 0.2765, 0.2763],
        [0.3329, 0.3194, 0.3122],
        [0.2757, 0.2700, 0.2698],
        [0.2946, 0.2590, 0.2552],
        [0.2573, 0.2244, 0.2192],
        [0.2883, 0.2876, 0.2743],
        [0.3901, 0.3336, 0.3133],
        [0.3318, 0.3071, 0.3050],
        [0.3168, 0.3134, 0.3055],
        [0.4896, 0.4417, 0.4329],
        [0.4752, 0.3161, 0.3030],
        [0.4314, 0.3725, 0.2921],
        [0.5200, 0.3816, 0.3425],
        [0.3337, 0.2814, 0.2695],
        [0.2969, 0.2955, 0.2869],
        [0.3390, 0.3373, 0.3125],
        [0.3718, 0.3698, 0.3374],
        [0.2926, 0.2915, 0.2646],
        [0.2979, 0.2950, 0.2909],
        [0.3237, 0.2924, 0.2719],
        [0.3617, 0.3616, 0.3308],
        [0.2742, 0.2678, 0.2626],
        [0.4197,

In [ ]:
nearest_scores = torch.max(similarity_matrix, dim=1).values

In [ ]:
from google.colab import drive
drive.mount('/content/drive')